# Beam Class Regression Tests
----

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from distgen import Generator
from distgen.physical_constants import MC2

In [3]:
input_file = "../examples/data/rad.gaussian.in.yaml"
gen = Generator(input_file,verbose=0)
beam = gen.beam()

coordinates = {'x', 'y', 'z', 'px', 'py', 'pz', 
               'r', 'theta', 'pr', 'ptheta', 
               'xp', 'yp', 'thetax', 'thetay',
               'gamma', 'energy', 'kinetic_energy', 
               'beta_x', 'beta_y', 'beta_z'}

# Statistical Tests

$\sum_i w_i = 1$

In [4]:
assert np.abs(np.sum(beam['w'])-1)<1e14, 'Beam weights not normalized'

$\langle \mathcal{O}\rangle = \sum_i w_i \mathcal{O}_i$

In [5]:
for var in coordinates:
    assert np.abs(beam.avg(var) - np.sum(beam['w']*beam[var])).magnitude<1e-14, f'<{var}> is not correct'

$\sigma_{\mathcal{O}}^2 = \sum_i w_i (\mathcal{O}_i-\langle \mathcal{O}\rangle)^2 $ 

In [6]:
for var in coordinates:
    assert np.abs(beam.std(var)**2 - np.sum(beam['w']*(beam[var]-beam.avg(var))**2)).magnitude<5e-10, f'<{var}> is not correct'

# Cylindrical Coordinates
---
# Getting

$r=\sqrt{ x^2 + y^2 }$

In [7]:
deviation =  np.abs(beam['r'] - np.sqrt(beam['x']**2 + beam['y']**2))
assert max(deviation.magnitude)<1e-14, 'Radial coordinate incorrect'

$x=r\cos\theta$

In [8]:
deviation = np.abs(beam['x'] - beam['r']*np.cos(beam['theta']))
assert max(deviation.magnitude)<1e-14, 'x coordinate incorrect'

$y = r\sin\theta$ 

In [9]:
deviation = np.abs(beam['x'] - beam['r']*np.cos(beam['theta']))
assert max(deviation.magnitude)<5e-13, 'y coordinate incorrect'

$p_r = p_x\cos\theta - p_y\sin\theta$

In [10]:
deviation = np.abs(beam['pr'] - beam['px']*np.cos(beam['theta']) - beam['py']*np.sin(beam['theta']))
assert max(deviation.magnitude)<5e-13, 'pr coordinate incorrect'

$p_{\theta} = -p_x\cos\theta + p_y\sin\theta$

In [11]:
deviation = np.abs(beam['ptheta'] + beam['px']*np.sin(beam['theta']) - beam['py']*np.cos(beam['theta']))
assert max(deviation.magnitude)<5e-13, 'ptheta coordinate incorrect'

# Transverse Derivatives and Angles
---

# Getting

$x^{\prime} = p_x/p_z$

In [12]:
deviation = np.abs(beam['xp'] - beam['px'].to(beam['pz'].units)/beam['pz'])
assert max(deviation.magnitude)<1e-14, 'xp coordinate incorrect'

$y^{\prime} = p_y/p_z$

In [13]:
deviation = np.abs(beam['yp'] - beam['py'].to(beam['py'].units)/beam['pz'])
assert max(deviation.magnitude)<1e-14, 'yp coordinate incorrect'

$\theta_x = \arctan(p_x/p_z)$

In [14]:
deviation = np.abs(beam['thetax'] - np.arctan(beam['px'].to(beam['px'].units)/beam['pz']))
assert max(deviation.magnitude)<1e-14, f'thetax coordinate deviation = {max(deviation)}'

$\theta_y = \arctan(p_y/p_z)$

In [15]:
deviation = np.abs(beam['thetay'] - np.arctan(beam['py'].to(beam['py'].units)/beam['pz']))
assert max(deviation.magnitude)<1e-14, 'thetay coordinate incorrect'

# Relativistic Quantities
---
$p=\sqrt{p_x^2 + p_y^2 + p_z^2}$

In [16]:
deviation = np.abs(beam['p'] - np.sqrt(beam['px']**2 +beam['py']**2 + beam['pz']**2))
assert max(deviation.magnitude)<1e-14, 'thetay coordinate incorrect'

$\gamma = \sqrt{1+\left(\frac{p}{mc}\right)^2}$

In [17]:
deviation = np.abs( beam['gamma'] - np.sqrt(1+ (beam['p'].to('eV/c').magnitude/MC2.magnitude)**2) )
assert max(deviation.magnitude)<1e-13, 'gamma coordinate incorrect'

$\beta_{x_i} = p_{x_i}/\gamma$

In [18]:
for var in ['x', 'y', 'z']:
    deviation = np.abs( beam[f'beta_{var}'] - beam[f'p{var}'].to('GB')/beam['gamma'])  
    assert max(deviation.magnitude)<1e-14, f'beta_{var} coordinate incorrect'

KE = $mc^2(\gamma-1)$

In [19]:
deviation = np.abs( beam['kinetic_energy'] - MC2*(beam['gamma']-1) ) 
assert max(deviation.magnitude)<1e-14, 'KE coordinate incorrect'

E = $mc^2\gamma$

In [20]:
deviation = np.abs( beam['energy'] - MC2*(beam['gamma']) ) 
assert max(deviation.magnitude)<1e-14, 'E coordinate incorrect'

# Twiss Parameters
---
# Getting

$\epsilon_{n,x_i} = \frac{1}{mc}\sqrt{\sigma_{x_i}^2\sigma_{p_{x_i}}^2 - \langle \left(x_i-\langle x_i\rangle\right)\left(p_{x_i}-\langle p_{x_i}\rangle\right)\rangle^2 }$

In [21]:
for var in ['x', 'y']:
    stdx = beam.std(var)
    stdp = (beam.std(f'p{var}').to('GB')).magnitude
    dx = beam[var]-beam.avg(var)
    dp = (beam[f'p{var}']-beam.avg(f'p{var}')).to('GB').magnitude
    
    deviation = np.abs(beam.emitt(var) - np.sqrt(stdx**2 * stdp**2 - (np.sum(beam['w']*dx*dp))**2 ))
    assert deviation.magnitude < 1e-14

$\epsilon_{x} = \sqrt{\sigma_x^2\sigma_{x^{\prime}}^2 - \langle \left(x-\langle x\rangle\right)\left(x^{\prime}-\langle x^{\prime}\rangle\right)\rangle^2 }$

In [22]:
for var in ['x', 'y']:
    stdx = beam.std(var)
    stdp = (beam.std(f'{var}p')).magnitude
    dx = beam[var]-beam.avg(var)
    dp = (beam[f'{var}p']-beam.avg(f'{var}p')).magnitude
    
    deviation = np.abs(beam.emitt(var,'geometric') - np.sqrt(stdx**2 * stdp**2 - (np.sum(beam['w']*dx*dp))**2 ))
    assert deviation.magnitude < 1e-14

Twiss $\beta_{x_i} = \frac{\sigma_x^2}{\epsilon_x}$

In [23]:
for var in ['x', 'y']:
    stdx = beam.std(var)
    epsx = beam.emitt(var,'geometric')

    deviation = np.abs( beam.Beta(var) - stdx**2/epsx )
    assert deviation.magnitude < 1e-14

Twiss $\alpha_{x_i} = -\frac{\langle(x-\langle x\rangle)(x^{\prime}-\langle x^{\prime}\rangle)\rangle}{\epsilon_x}$

In [24]:
for var in ['x', 'y']:
    dx = beam[var]-beam.avg(var)
    dp = (beam[f'{var}p']-beam.avg(f'{var}p'))
    epsx = beam.emitt(var,'geometric')
    deviation = np.abs( beam.Alpha(var) + sum(beam['w']*dx*dp)/epsx )
    assert deviation.magnitude < 1e-15

Twiss $\gamma_{x_i} = \frac{\sigma_{x^{\prime}}^2}{\epsilon_x}$

In [25]:
for var in ['x', 'y']:
    stdp = beam.std(f'{var}p')
    epsx = beam.emitt(var,'geometric')
    deviation = np.abs( beam.Gamma(var) - stdp**2/epsx )
    assert deviation.magnitude < 5e-15